In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## File Descriptions and Data Field Information

### train.csv
- The training data, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.
- store_nbr identifies the store at which the products are sold.
- family identifies the type of product sold.
- sales gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
- onpromotion gives the total number of items in a product family that were being promoted at a store at a given date.

In [2]:
train = pd.read_csv('train.csv', parse_dates=['date'])
train.dtypes

id                      int64
date           datetime64[ns]
store_nbr               int64
family                 object
sales                 float64
onpromotion             int64
dtype: object

In [3]:
train.groupby('store_nbr').count().describe()

,id,date,family,sales,onpromotion
count,54.0,54.0,54.0,54.0,54.0
mean,55572.0,55572.0,55572.0,55572.0,55572.0
std,0.0,0.0,0.0,0.0,0.0
min,55572.0,55572.0,55572.0,55572.0,55572.0
25%,55572.0,55572.0,55572.0,55572.0,55572.0
50%,55572.0,55572.0,55572.0,55572.0,55572.0
75%,55572.0,55572.0,55572.0,55572.0,55572.0
max,55572.0,55572.0,55572.0,55572.0,55572.0


### Each store has 55572 entries

In [4]:
train.groupby(['store_nbr', 'family']).count().describe()

,id,date,sales,onpromotion
count,1782.0,1782.0,1782.0,1782.0
mean,1684.0,1684.0,1684.0,1684.0
std,0.0,0.0,0.0,0.0
min,1684.0,1684.0,1684.0,1684.0
25%,1684.0,1684.0,1684.0,1684.0
50%,1684.0,1684.0,1684.0,1684.0
75%,1684.0,1684.0,1684.0,1684.0
max,1684.0,1684.0,1684.0,1684.0


## Total 1782 categories, or need to create 1782 models for each category

In [5]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


### MIssing Days if Any

In [6]:
from datetime import date
import numpy as np

d0 = np.sort(train['date'])[0]
d1 = np.sort(train['date'])[-1]
delta = d1 - d0
no_of_days = train['date'].nunique()
print('The number of days between the given range of dates is :')
print(delta.astype('timedelta64[D]')+1)
print(no_of_days)

The number of days between the given range of dates is :
1688 days
1684


In [7]:
d1

numpy.datetime64('2017-08-15T00:00:00.000000000')

### Find the missing Dates

In [8]:
pd.date_range(start=d0, end=d1).difference(train['date'].unique())

DatetimeIndex(['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25'], dtype='datetime64[ns]', freq=None)

#### Conclusion: In Christmas, stores are always closed 

### Total Sales across Family

In [ ]:
sns.set(rc={'figure.figsize':(20,20)})
g= sns.catplot(data=train, x='family', y='sales', kind='box', whis=[5,95], sym="", height=15, aspect=1/1
            # col=year(date), 
            # col_wrap=3
           )

g.fig.suptitle("Total Sales across Family", y=1.03)
plt.xticks(rotation=90)
plt.show()

### Total Sales across Store Number

In [ ]:
print("Total stores are", train['store_nbr'].nunique())

In [ ]:
# TBD

### Total Sales & Onpromotion

In [ ]:
# TBD

### Time Series of Total Sales based on resampling on mean

In [ ]:
sns.set(rc={'figure.figsize':(12,12)})

resampled_data = train.groupby('date', as_index=False)['sales'].sum().resample('W', on='date').mean()
g = sns.relplot(x="date", y="sales", data=resampled_data, kind="line")

g.fig.suptitle("Total Sales across Time", y=1.03)

In [ ]:
sns.set(rc={'figure.figsize':(12,12)})

resampled_data = train[['date', 'sales']].resample('W', on='date').mean()
g = sns.relplot(x="date", y="sales", data=resampled_data, kind="line")

g.fig.suptitle("Total Sales across Time", y=1.03)

### Singularity for Total Sales

In [ ]:
import statsmodels.tsa.stattools as sts
sts.adfuller(train.groupby('date', as_index=False)['sales'].sum()['sales'])

#### Conclusion: Not Stationary but close to stationatity

In [ ]:
## TBD

### Seasonality

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
intermediate = pd.DataFrame(train.groupby('date')['sales'].sum()).asfreq('d')
intermediate.fillna(0, inplace=True)

In [ ]:
s_dec_additive = seasonal_decompose(intermediate['sales'] , model='additive')
s_dec_additive.plot()
plt.show()

### ACF & PACF of Total Sales Time Series

In [ ]:
import statsmodels.graphics.tsaplots as sgt

sgt.plot_acf(train.groupby('date', as_index=False)['sales'].sum()['sales'], lags=40, zero=False)
plt.title("ACF For Prices", size=20)
plt.show()

#### Conclusion: 
- Dependence on weekends
- Need separate tests for Weekdays & Weeknends

In [ ]:
sgt.plot_pacf(train.groupby('date', as_index=False)['sales'].sum()['sales'], lags=40, zero=False, method='ols')
plt.title("PACF For Prices", size=20)
plt.show()

### Time Series of Total Sales after May 2017

In [ ]:
sns.set(rc={'figure.figsize':(12,12)})

resampled_data = train[train['date']>='2017-06-01']
g = sns.relplot(x="date", y="sales", data=resampled_data, kind="line", errorbar=None)

g.fig.suptitle("Total Sales across Time after May 2017", y=1.03)
plt.xticks(rotation=90)
plt.show()

## Last Date in Train

In [ ]:
max(train['date'])

### Weekdays & Weekends

In [ ]:
by_weekday = train.groupby(train['date'].dt.dayofweek)[['sales', 'onpromotion']].mean()
by_weekday.index = ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']


In [ ]:
import numpy as np
weekend = np.where(train['date'].dt.weekday < 5, 'Weekday', 'Weekend')
by_weekend = train.groupby(weekend)[['sales', 'onpromotion']].mean()

by_weekend

### test.csv
- The test data, having the same features as the training data. You will predict the target sales for the dates in this file.
- The dates in the test data are for the 15 days after the last date in the training data.

In [ ]:
test = pd.read_csv('test.csv', parse_dates=['date'])
test.dtypes

In [ ]:
test.head()

In [ ]:
test.groupby('store_nbr').count().describe()

### Test data set is from 16th August to 31st August

### sample_submission.csv
- A sample submission file in the correct format.

### stores.csv
- Store metadata, including city, state, type, and cluster.
- cluster is a grouping of similar stores.

In [ ]:
stores = pd.read_csv('stores.csv')
stores.dtypes

In [ ]:
stores.head()

In [ ]:
stores.shape

In [ ]:
stores.cluster.nunique()

In [ ]:
stores.city.unique()

In [ ]:
stores.state.unique()

### oil.csv
- Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

In [ ]:
oil = pd.read_csv('oil.csv', parse_dates=['date'])
oil.dtypes

In [ ]:
oil.head()

In [ ]:
sns.set(rc={'figure.figsize':(12,12)})

resampled_data = oil.resample('W', on='date').mean()
g = sns.relplot(x="date", y="dcoilwtico", data=resampled_data, kind="line")

g.fig.suptitle("Oil price across Time", y=1.03)
plt.show()

### MIssing Days if Any

In [ ]:
from datetime import date
import numpy as np

d0 = np.sort(oil['date'])[0]
d1 = np.sort(oil['date'])[-1]
delta = d1 - d0
no_of_days = oil['date'].nunique()
print('The number of days between the given range of dates is :')
print(delta.astype('timedelta64[D]')+1)
print(no_of_days)

In [ ]:
d1

### Find the missing Dates

In [ ]:
pd.date_range(start=d0, end=d1).difference(oil['date'].unique())

In [ ]:
oil.set_index("date", inplace=True)
oil=oil.asfreq('d')
oil=oil.fillna(method='ffill')

#### Conclusion
- oil prices are unavailable on weekends
- Use ffill to fill oil prices for weekends

In [ ]:
from datetime import date
import numpy as np

d0 = np.sort(oil.index)[0]
d1 = np.sort(oil.index)[-1]
delta = d1 - d0
no_of_days = oil.index.nunique()
print('The number of days between the given range of dates is :')
print(delta.astype('timedelta64[D]')+1)
print(no_of_days)

### Correlation between Oil & Sales

In [ ]:
oil_sales = pd.DataFrame(train.groupby('date')['sales'].sum()).merge(oil, how='inner', left_index=True, right_index=True)

In [ ]:
oil_sales.corr()

#### Conclusion: Negative Correlation on oil prices and Sum of Sales on a given day

### holidays_events.csv

- Holidays and Events, with metadata
**NOTE**: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.

- Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

- Additional Notes
 - Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
 - A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

In [ ]:
holidays = pd.read_csv('holidays_events.csv', parse_dates=['date'])
holidays.dtypes

In [ ]:
holidays.type.unique()

In [ ]:
holidays[holidays['description']=="Independencia de Guayaquil"]


### Event, Holidays that are not transferred, Additional & Bridge are marked as holidays

In [ ]:
holidays['is_holiday'] = np.where((holidays['transferred']) | (holidays['type']=='Work Day') , 0, 1)

In [ ]:
holidays[holidays['description']=="Independencia de Guayaquil"]


In [ ]:
locale_array = {}
for item in holidays.locale_name.unique():
    if item == 'Ecuador':
        locale_array[item]='All'
    elif item in stores.city.unique():
        locale_array[item]='City'
    elif item in stores.state.unique():
        locale_array[item]='State'
    else:
        locale_array[item]="None of the Above"

In [ ]:
locale_array

## Transactions

In [ ]:
transactions = pd.read_csv('transactions.csv', parse_dates=['date'])
transactions.dtypes

In [ ]:
transactions.tail()

### Dependence on Payday

In [ ]:
payday = pd.read_csv('train.csv', parse_dates=['date'])
payday.dtypes

In [ ]:
import datetime 
def is_payday(dt):
    todays_month = dt.month
    tomorrows_month = (dt + datetime.timedelta(days=1)).month
    return int((tomorrows_month != todays_month) | (dt.day==15))

In [ ]:
payday['is_payday'] = payday['date'].apply(is_payday)

In [ ]:
payday[payday['date']=='2013-01-31']

# Building Simple Multiple Regression Model
- Build 1782 models 
- Model will depend on holiday(includes weekend), oil price & payday

In [ ]:
df_train = pd.read_csv('train.csv', parse_dates=['date'])
import datetime 
def is_payday(dt):
    todays_month = dt.month
    tomorrows_month = (dt + datetime.timedelta(days=1)).month
    return int((tomorrows_month != todays_month) | (dt.day==15))

df_train['is_payday'] = df_train['date'].apply(is_payday)

df_train['is_weekend'] = np.where(train['date'].dt.weekday < 5, 0, 1)


df_train.head()

In [ ]:
df_holidays = pd.read_csv('holidays_events.csv', parse_dates=['date'])
df_holidays['is_holiday'] = np.where((df_holidays['transferred']) | (df_holidays['type']=='Work Day') , 0, 1)

df_holidays.head()

In [ ]:
df_oil = pd.read_csv('oil.csv', parse_dates=['date'])
df_oil.set_index("date", inplace=True)
df_oil=df_oil.asfreq('d')
df_oil=df_oil.fillna(method='ffill')

df_oil.reset_index(inplace=True)

df_oil=df_oil.fillna(method='bfill')

df_oil.head()

In [ ]:
df_stores = pd.read_csv('stores.csv')

locale_array = {}
for item in df_holidays.locale_name.unique():
    if item == 'Ecuador':
        locale_array[item]=[item]
    elif item in stores.city.unique():
        locale_array[item]=[item]
    elif item in stores.state.unique():
        locale_array[item]=[item]
    else:
        locale_array[item]=[None]
        
def mapping_terr(place):
    if place not in locale_array:
        return ''
    else:
        return locale_array[place]



In [ ]:
df_mapping_file = pd.DataFrame(locale_array)
df_mapping_file = df_mapping_file.transpose()

df_mapping_file.columns=['place']
df_mapping_file

In [ ]:
df_temp_2=df_stores.merge(df_mapping_file, how='left', left_on='city', right_index=True)
df_temp_2.head()

In [ ]:
df_temp_3=df_stores.merge(df_mapping_file, how='left', left_on='state', right_index=True)
df_temp_3.head()

In [ ]:
df_temp = df_stores
df_temp['place']='Ecuador'

df_store_mapping = pd.concat([df_temp_2, 
                              df_temp_3,
                              df_temp], ignore_index=True)

In [ ]:
df_store_mapping = df_store_mapping[df_store_mapping['place'].notna()]


In [ ]:
df_holidays_store = df_holidays[['date', 'is_holiday', 'locale_name']].merge(df_store_mapping,
                                                                             how='left',
                                                                             left_on='locale_name',
                                                                             right_on='place')

In [ ]:
df_holidays_store = df_holidays_store[['date', 'is_holiday', 'store_nbr']]

df_holidays_store = df_holidays_store.groupby(['date', 'store_nbr'], as_index=False)['is_holiday'].sum()

df_holidays_store['is_holiday'] = np.where(df_holidays_store['is_holiday']==0, 0, 1)

In [ ]:
df_holidays_store.head()

In [ ]:
df_train_final = df_train.merge(df_holidays_store, how='left', left_on=['date', 'store_nbr'], right_on=['date', 'store_nbr']).merge(df_oil, left_on='date', right_on='date', how='left')

In [ ]:
df_train_final.is_holiday = np.where(df_train_final.is_holiday.fillna(0)+df_train_final.is_weekend==0, 0, 1)

In [ ]:
df_train_final = df_train_final.drop('is_weekend', axis=1)
df_train_final.head()

In [ ]:
df_train_final[df_train_final['date']=='2013-01-02']

## Training the model

In [ ]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as MSE

In [ ]:
df_1_1 = df_train_final[(df_train_final['store_nbr']==1) & (df_train_final['family']=='AUTOMOTIVE')]
X = df_1_1[['onpromotion','is_payday','is_holiday', 'dcoilwtico' ]]
y = df_1_1['sales']

model=LinearRegression()
model.fit(X,y)


In [ ]:
np.sqrt(MSE(y, model.predict(X)))

## ARIMA models

In [ ]:
from pmdarima.arima import auto_arima

In [ ]:
# model_auto = auto_arima(df_1_1.sales, exogenous=X,
#                       m = 5, max_p = 5, max_q = 5, max_P = 5, max_Q = 5)

## Random Forests

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=15, random_state=0)
regr.fit(X, y)


In [ ]:
r2_score(y, regr.predict(X))

## Production ready predictions

### Making Test Set Ready

In [ ]:
df_test = pd.read_csv('test.csv', parse_dates=['date'])
import datetime 
def is_payday(dt):
    todays_month = dt.month
    tomorrows_month = (dt + datetime.timedelta(days=1)).month
    return int((tomorrows_month != todays_month) | (dt.day==15))

df_test['is_payday'] = df_test['date'].apply(is_payday)

df_test['is_weekend'] = np.where(df_test['date'].dt.weekday < 5, 0, 1)

df_test.head()

In [ ]:
df_test_final = df_test.merge(df_holidays_store, how='left', left_on=['date', 'store_nbr'], right_on=['date', 'store_nbr']).merge(df_oil, left_on='date', right_on='date', how='left')
df_test_final.is_holiday = np.where(df_test_final.is_holiday.fillna(0)+df_test_final.is_weekend==0, 0, 1)


df_test_final = df_test_final.drop('is_weekend', axis=1)
df_test_final.head()

In [ ]:
# df_train = df_train.asfreq('D', method='bfill')
# df_train 

# df_train['weekshift'] = df_trai

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

final_file = pd.DataFrame()
count = 0

for store_nbr in df_test_final.store_nbr.unique():
    for family in df_test_final.family.unique():
        df_1_1 = df_train_final[(df_train_final['store_nbr']==store_nbr) & (df_train_final['family']==family)]
        
#         df_1_1 = df_1_1.asfreq('D', method='bfill')
#         df_1_1['shifted_by_week'] = df_1_1.shift(periods=-7, freq="D") 
        
#         df_1_1.reset_index()
        
        X = df_1_1[['onpromotion','is_payday','is_holiday', 'dcoilwtico' , 'date', 'sales']]
        X = X.set_index('date')
        X = X.asfreq('D', method='bfill')
        y = X['sales']
        
        model = SARIMAX(y, exog=X[['onpromotion','is_payday','is_holiday', 'dcoilwtico']], order=(1,1,1),
                       seasonal_order=(2,0,0,7))
        arima = model.fit(maxiter=100)

        
        test = df_test_final[(df_test_final['store_nbr']==store_nbr) & (df_test_final['family']==family)]
        
#         test = test.asfreq('D', method='bfill')
#         test['shifted_by_week'] = test.shift(periods=-7, freq="D") 
        
#         df_1_1.reset_index()
        
        start_date='2017-08-16'
        end_date='2017-08-31'
        
        test = test.set_index('date')        
        test = test.asfreq('D', method='bfill')
        
        new = pd.concat([test.id, pd.Series(arima.predict(exog=test[['onpromotion','is_payday','is_holiday', 'dcoilwtico' ]],
                                                         start=start_date, end=end_date), 
                                            index=test.index)],axis=1)
        final_file = pd.concat([final_file, new])
        
        count +=1
        print(count)

In [ ]:
final_file.sort_index().to_csv('submission_4.csv', index=False, header=['id', 'sales'])